In [4]:
path = ""
learning_rate = 0.05
Batch_Size = 30
iterations =5
show_graph = 0
save_model =1
do_testing = 1
run_prediction = 1
model_load = 1
Test_size = 2/14

In [2]:
from imports import *

# criterion = nn.MSELoss()
net = convnet().cuda()
optimizer = optim.SGD(net.parameters(), lr=learning_rate,momentum=0.9)
df = pd.read_csv(os.path.join(path, "training.csv"))
train, test = train_test_split(df, test_size=Test_size)
data = dataset(df = train , root_dir = os.path.join(path, "images"))
dataloader = DataLoader(data, batch_size=Batch_Size, shuffle=True, num_workers=0)
data_test = dataset(df = test , root_dir = os.path.join(path, "images"))
dataloader_test = DataLoader(data_test, batch_size=Batch_Size, shuffle=True, num_workers=0)

In [3]:
if model_load:
    net=convnet().cuda()
    net.load_state_dict(torch.load(os.path.join("weight","tmpgc4epqm7")))
    net.eval()

In [ ]:
for epoch in range(iterations):
    runningLoss = 0 
    count=0
    net.train(True)
    for data in dataloader:
        inputs = data["image"].cuda()
        labels = data["box"].type(torch.DoubleTensor).cuda()
        inputs = Variable(inputs)
        labels = Variable(labels)
        optimizer.zero_grad()   
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss.data.item()
        count+=1
        print(loss.data.item())
    if save_model:
        with tempfile.NamedTemporaryFile(dir = os.path.join(path+"weight"), delete = False) as f:
            torch.save(net.state_dict(), f)    
    avgTrainLoss = runningLoss/count
    
    
    print("Testing-----------------------------------------------------------------------------------------")
    print("Testing-----------------------------------------------------------------------------------------")
    print("Testing-----------------------------------------------------------------------------------------")
    if do_testing:
        with torch.no_grad():
            running_correct = 0
            count1=0
            for data in dataloader_test:
                inputs = data["image"].cuda()
                labels = data["box"].type(torch.DoubleTensor).cuda()
                inputs = Variable(inputs)
                labels = Variable(labels)
                outputs = net(inputs)
                running_correct += iou(outputs,labels).item()
                count1+=1
            avgTestAcc = float(running_correct)/count1
            print(avgTestAcc)

48.95482040490344
51.85074576305308
54.55948285792881
47.10284874663875
49.8726002457372
50.25854592518253
53.35859000999143
45.96274275511016
48.36715172779818
52.86421106225915
53.18129450619778
51.381485072879975
54.07340991922338
51.14999359960499
54.782220639867305
46.743731031855035
47.07352401429972
50.35621846261051
54.0966458427976
50.22248672606969
44.91831990361299
50.25405043052878
50.471524033104274
51.40599857135699
43.81514139680483
50.65712157538995
49.27155199240038
53.373882893653395
46.75401246515109
51.47437858204728
48.3971593420984
50.99104132030736
48.604127314836
51.95002616301218
47.080439968642
42.94439245282453
51.39474066284242
48.93955908589132
49.27997449341848
54.50650626491475
52.199339915876195
49.536606139828066
50.71124803951212
50.13186103462407
50.40167968275185
53.53312230045636
47.566272751133205
49.618108708469975
47.465491721925275
52.35270077741855
51.62535426985079
51.18333317464101
50.727676803836495
46.71136302641162
51.685100006569286
46.44

In [12]:

if run_prediction:
    import csv
    predict_set = pd.read_csv(os.path.join(path , "test.csv"))
    f = open(path+"out1.csv", "w")
    writer = csv.writer(f)
    row = []
    row.append(["image_name","x1", "x2", "y1", "y2"])
    with torch.no_grad():
        for index, data in predict_set.iterrows():
                image = cv2.imread(os.path.join(path, "images/"+data[0]))
                edges = cv2.Canny(image,100,200)
                kernel = np.ones((5,5), np.uint8)
                img_dilation = cv2.dilate(edges, kernel, iterations=1)
                img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
                img1 = cv2.medianBlur(img_erosion,3)
                img_dilation = cv2.dilate(img1, kernel, iterations=1)
                img2=img_dilation.reshape(1,640, 480)
                image = image.reshape(3,640,480)
                image = np.concatenate((image, img2), axis = 0)
                image = image.reshape(1,4,640,480)
                image = torch.from_numpy(image)
                inputs = Variable(image.type(torch.DoubleTensor).cuda())
                outputs = net(inputs)
                row.append([data["image_name"], outputs[0][0].item(), outputs[0][2].item(), outputs[0][1].item(), outputs[0][3].item()])
    writer.writerows(row)
    f.close()

    

In [6]:
print("Done")

Done


In [7]:
with tempfile.NamedTemporaryFile(dir = os.path.join(path+"weight"), delete = False) as f:
    torch.save(net.state_dict(), f)